## Lab08-4

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.5
batch_size = 10

In [4]:
# mnist dataset
mnist_train = dsets.MNIST(root="MNIST_data/",
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root="MNIST_data/",
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
w1 = torch.nn.Parameter(torch.Tensor(784,30)).to(device)
b1 = torch.nn.Parameter(torch.Tensor(30)).to(device)
w2 = torch.nn.Parameter(torch.Tensor(30,10)).to(device)
b2 = torch.nn.Parameter(torch.Tensor(10)).to(device)

In [7]:
torch.nn.init.normal_(w1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(w2)
torch.nn.init.normal_(b2)

Parameter containing:
tensor([ 0.3078, -1.9857,  1.0512,  1.5122, -1.0199, -0.7402, -1.3111,  0.6142,
        -0.6474,  0.1758], requires_grad=True)

In [8]:
def sigmoid(x):
    return 1.0/(1.0+torch.exp(-x))

In [9]:
def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

In [12]:
x_test = mnist_test.test_data.view(-1,28*28).float().to(device)[:1000]
y_test = mnist_test.test_labels.to(device)[:1000]
i=0
while not i == 10000:
    for x, y in data_loader:
        i += 1
        
        # forward
        x = x.view(-1,28*28).to(device)
        y = torch.zeros((batch_size,10)).scatter_(1,y.unsqueeze(1),1).to(device) # one-hot
        l1 = torch.add(torch.matmul(x,w1),b1)
        a1 = sigmoid(l1)
        l2 = torch.add(torch.matmul(a1,w2),b2)
        y_pred = sigmoid(l2)
        
        diff = y_pred - y
        
        # Back prop (chain rule)
        
        d_l2 = diff*sigmoid_prime(l2)
        d_b2 = d_l2
        d_w2 = torch.matmul(torch.transpose(a1,0,1), d_l2)
        
        d_a1 = torch.matmul(d_l2, torch.transpose(w2,0,1))
        d_l1 = d_a1*sigmoid_prime(l1)
        d_b1 = d_l1
        d_w1 = torch.matmul(torch.transpose(x,0,1),d_l1)
        
        w1 = w1 - learning_rate*d_w1
        b1 = b1 - learning_rate*torch.mean(d_b1,0)
        w2 = w2 - learning_rate*d_w2
        b2 = b2 - learning_rate*torch.mean(d_b2,0)
        
        if i % 1000 == 0:
            l1 = torch.add(torch.matmul(x_test,w1),b1)
            a1 = sigmoid(l1)
            l2 = torch.add(torch.matmul(a1,w2),b2)
            y_pred = sigmoid(l2)
            acct_mat = torch.argmax(y_pred,1) == y_test
            acct_res = acct_mat.sum()
            print(acct_res.item())
            
        if i == 10000:
            break

819
862
866
864
890
899
906
889
912
909
